# 🧠 Cogaly - Early Alzheimer's Risk Detection

**AI 4 Alzheimer's Hackathon Submission**

This notebook implements an XGBoost classifier with SHAP explainability for early Alzheimer's disease risk detection.

## Features
- High accuracy (94.42%) on test set
- SHAP-based interpretable predictions
- Complete training and evaluation pipeline
- Production-ready model export

---

## Step 1: Install Required Packages

Run this cell first to install all dependencies.

In [ ]:
# Uncomment and run this cell in Google Colab
# !pip install pandas numpy scikit-learn xgboost shap matplotlib seaborn -q
# print("✅ All packages installed successfully!")

## Step 2: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve
)
import xgboost as xgb
import shap
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("🧠 COGALY - ALZHEIMER'S RISK DETECTION SYSTEM")
print("=" * 80)
print("\n✅ All libraries imported successfully!")

## Step 3: Load Dataset

**Option 1:** Upload the dataset directly in Colab  
**Option 2:** Mount Google Drive  
**Option 3:** Use the local path (for local environments)

In [ ]:
# === OPTION 1: Upload file directly in Colab ===
# from google.colab import files
# uploaded = files.upload()
# data_path = 'alzheimers.csv'

# === OPTION 2: Load from Google Drive ===
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = '/content/drive/MyDrive/path/to/alzheimers.csv'

# === OPTION 3: Local path ===
data_path = '../data/alzheimers.csv'  # Change this path as needed

print("📁 Data path configured:", data_path)

## Step 4: Load and Explore Data

In [ ]:
print("\n" + "=" * 80)
print("📊 LOADING DATASET")
print("=" * 80)

df = pd.read_csv(data_path)

print(f"✅ Dataset loaded successfully!")
print(f"   • Total samples: {df.shape[0]:,}")
print(f"   • Total features: {df.shape[1]}")

df.head()

In [ ]:
print("📈 Target Distribution:")
target_dist = df['Diagnosis'].value_counts()
print(target_dist)
print(f"\nClass Balance (%):\n{df['Diagnosis'].value_counts(normalize=True) * 100}")

In [ ]:
# Visualize target distribution
plt.figure(figsize=(8, 5))
colors = ['#2ecc71', '#e74c3c']
target_dist.plot(kind='bar', color=colors, edgecolor='black')
plt.title('Target Distribution: Alzheimer\'s Diagnosis', fontsize=14, fontweight='bold')
plt.xlabel('Diagnosis (0 = No Risk, 1 = At Risk)')
plt.ylabel('Count')
plt.xticks(rotation=0)
for i, v in enumerate(target_dist):
    plt.text(i, v + 20, f'{v}', ha='center', fontsize=12)
plt.tight_layout()
plt.show()

## Step 5: Data Preprocessing

In [ ]:
print("\n" + "=" * 80)
print("🔧 DATA PREPROCESSING")
print("=" * 80)

# Remove non-predictive columns
columns_to_drop = ['PatientID', 'DoctorInCharge']
df_processed = df.drop(columns=columns_to_drop, errors='ignore')
print(f"✅ Removed columns: {columns_to_drop}")

# Separate features and target
X = df_processed.drop('Diagnosis', axis=1)
y = df_processed['Diagnosis']
print(f"✅ Features: {X.shape[1]}, Target: Diagnosis")

# Handle missing values
missing_count = X.isnull().sum().sum()
if missing_count > 0:
    print(f"⚠️  Handling {missing_count} missing values...")
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if X[col].isnull().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)
    
    categorical_cols = X.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if X[col].isnull().sum() > 0:
            X[col].fillna(X[col].mode()[0], inplace=True)
    print(f"✅ Missing values handled")
else:
    print(f"✅ No missing values detected")

# Encode categorical features
categorical_cols = X.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    print(f"✅ Encoding {len(categorical_cols)} categorical features...")
    X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print(f"\n✅ Preprocessing complete! Final features: {X.shape[1]}")

## Step 6: Train-Test Split (80/20)

In [ ]:
print("\n" + "=" * 80)
print("✂️  SPLITTING DATA (80/20)")
print("=" * 80)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"✅ Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

## Step 7: Feature Scaling

In [ ]:
print("\n" + "=" * 80)
print("📏 FEATURE SCALING")
print("=" * 80)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train), 
    columns=X_train.columns
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test), 
    columns=X_test.columns
)

print(f"✅ Features scaled (mean=0, std=1)")

## Step 8: Train XGBoost Model

In [ ]:
print("\n" + "=" * 80)
print("🤖 TRAINING XGBOOST CLASSIFIER")
print("=" * 80)

model = xgb.XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

print("⏳ Training in progress...")
model.fit(X_train_scaled, y_train)
print("✅ Model training completed!")

## Step 9: Make Predictions

In [ ]:
print("\n" + "=" * 80)
print("🔮 MAKING PREDICTIONS")
print("=" * 80)

y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)

print(f"✅ Predictions generated for {len(y_test):,} samples")

## Step 10: Model Evaluation

In [ ]:
print("\n" + "=" * 80)
print("📊 MODEL EVALUATION")
print("=" * 80)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"\n🎯 PERFORMANCE METRICS:")
print(f"{'='*60}")
print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   ROC-AUC:   {roc_auc:.4f}")
print(f"   Precision: {precision:.4f} ({precision*100:.2f}%)")
print(f"   Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"{'='*60}")

In [ ]:
print("\n📋 CLASSIFICATION REPORT:")
print(classification_report(y_test, y_pred, 
                          target_names=['No Alzheimer\'s', 'Alzheimer\'s']))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"\n🔢 CONFUSION MATRIX:")
print(f"   True Negatives:  {cm[0,0]:,}")
print(f"   False Positives: {cm[0,1]:,}")
print(f"   False Negatives: {cm[1,0]:,}")
print(f"   True Positives:  {cm[1,1]:,}")

# Visualize Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['No Risk', 'Risk'],
           yticklabels=['No Risk', 'Risk'],
           annot_kws={'size': 16})
plt.title('Confusion Matrix - Cogaly Model', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba[:, 1])
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
        label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
        label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Cogaly Model', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curve.png', dpi=150)
plt.show()

## Step 11: SHAP Explainability Analysis

In [ ]:
print("\n" + "=" * 80)
print("🔍 SHAP EXPLAINABILITY ANALYSIS")
print("=" * 80)

print("⏳ Computing SHAP values (this may take a moment)...")
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_scaled)
print("✅ SHAP values computed!")

In [ ]:
# Feature importance
shap_importance = pd.DataFrame({
    'Feature': X_test_scaled.columns,
    'Importance': np.abs(shap_values).mean(axis=0)
}).sort_values('Importance', ascending=False)

print(f"\n🏆 TOP 10 CONTRIBUTING FEATURES:")
print("-" * 60)
for idx, (_, row) in enumerate(shap_importance.head(10).iterrows(), 1):
    print(f"   {idx:2d}. {row['Feature']:30s} | {row['Importance']:.6f}")

In [ ]:
# SHAP Bar Plot
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_scaled, plot_type="bar", show=False)
plt.title('SHAP Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# SHAP Summary Plot
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test_scaled, show=False)
plt.title('SHAP Impact on Predictions', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_summary_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Step 12: Prediction Function with Explainability

In [ ]:
def predict_alzheimer_risk(patient_data, model, scaler, explainer, feature_cols):
    """
    Predict Alzheimer's risk with explainability
    
    Parameters:
    -----------
    patient_data : dict or pd.DataFrame
        Patient features
    model : XGBClassifier
        Trained XGBoost model
    scaler : StandardScaler
        Fitted feature scaler
    explainer : shap.TreeExplainer
        SHAP explainer for interpretability
    feature_cols : list
        List of feature names
    
    Returns:
    --------
    dict : Prediction results with explanations
    """
    if isinstance(patient_data, dict):
        patient_data = pd.DataFrame([patient_data])
    
    # Ensure all features present
    for col in feature_cols:
        if col not in patient_data.columns:
            patient_data[col] = 0
    
    patient_data = patient_data[feature_cols]
    
    # Scale and predict
    patient_scaled = pd.DataFrame(
        scaler.transform(patient_data), 
        columns=feature_cols
    )
    
    prediction = model.predict(patient_scaled)[0]
    probability = model.predict_proba(patient_scaled)[0]
    
    # SHAP values
    shap_vals = explainer.shap_values(patient_scaled)
    
    feature_contrib = pd.DataFrame({
        'Feature': feature_cols,
        'SHAP_Value': shap_vals[0],
        'Value': patient_scaled.values[0]
    }).sort_values('SHAP_Value', key=abs, ascending=False)
    
    top_features = feature_contrib.head(5)
    
    return {
        'predicted_class': int(prediction),
        'diagnosis': 'Alzheimer\'s Risk' if prediction == 1 else 'No Risk',
        'risk_score': float(probability[1]),
        'confidence': float(max(probability) * 100),
        'top_features': [
            {
                'feature': row['Feature'],
                'shap_value': float(row['SHAP_Value']),
                'impact': 'Increases Risk' if row['SHAP_Value'] > 0 else 'Decreases Risk'
            }
            for _, row in top_features.iterrows()
        ]
    }

print("✅ Prediction function defined!")

In [ ]:
# Test prediction on sample patient
print("\n" + "=" * 80)
print("🧪 TESTING PREDICTION FUNCTION")
print("=" * 80)

sample = X_test.iloc[0:1].copy()
result = predict_alzheimer_risk(sample, model, scaler, explainer, X_train.columns.tolist())

print(f"\n📋 SAMPLE PREDICTION:")
print(f"   Diagnosis: {result['diagnosis']}")
print(f"   Risk Score: {result['risk_score']:.4f} ({result['risk_score']*100:.2f}%)")
print(f"   Confidence: {result['confidence']:.2f}%")
print(f"\n   Top Contributing Features:")
for i, feat in enumerate(result['top_features'], 1):
    print(f"      {i}. {feat['feature']:25s} | {feat['impact']}")

## Step 13: Save Model and Artifacts

In [ ]:
print("\n" + "=" * 80)
print("💾 SAVING MODEL AND ARTIFACTS")
print("=" * 80)

# Save model
with open('cogaly_xgb_v1.pkl', 'wb') as f:
    pickle.dump(model, f)
print("✅ Model saved: cogaly_xgb_v1.pkl")

# Save scaler
with open('cogaly_scaler_v1.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✅ Scaler saved: cogaly_scaler_v1.pkl")

# Save feature columns
with open('cogaly_features_v1.pkl', 'wb') as f:
    pickle.dump(X_train.columns.tolist(), f)
print("✅ Features saved: cogaly_features_v1.pkl")

# Save explainer
with open('cogaly_explainer_v1.pkl', 'wb') as f:
    pickle.dump(explainer, f)
print("✅ Explainer saved: cogaly_explainer_v1.pkl")

# Save metrics
metrics = {
    'accuracy': float(accuracy),
    'roc_auc': float(roc_auc),
    'precision': float(precision),
    'recall': float(recall),
    'n_features': X_train.shape[1]
}

with open('cogaly_metrics_v1.pkl', 'wb') as f:
    pickle.dump(metrics, f)
print("✅ Metrics saved: cogaly_metrics_v1.pkl")

## Step 14: Download Files (For Google Colab)

In [ ]:
# Uncomment to download files in Google Colab:

# from google.colab import files
# files.download('cogaly_xgb_v1.pkl')
# files.download('cogaly_scaler_v1.pkl')
# files.download('cogaly_features_v1.pkl')
# files.download('cogaly_explainer_v1.pkl')
# files.download('cogaly_metrics_v1.pkl')
# files.download('confusion_matrix.png')
# files.download('roc_curve.png')
# files.download('shap_feature_importance.png')
# files.download('shap_summary_plot.png')

print("\n✅ To download in Colab, uncomment the download code above")

## 🎉 Training Complete!

### Model Summary

In [ ]:
print("\n" + "=" * 80)
print("🎉 COGALY MODEL TRAINING COMPLETE!")
print("=" * 80)
print(f"""
📊 MODEL SUMMARY:
   • Algorithm: XGBoost Classifier
   • Features: {X_train.shape[1]}
   • Training Samples: {len(X_train):,}
   • Test Samples: {len(X_test):,}

🎯 PERFORMANCE:
   • Accuracy:  {accuracy*100:.2f}%
   • ROC-AUC:   {roc_auc:.4f}
   • Precision: {precision*100:.2f}%
   • Recall:    {recall*100:.2f}%

💾 SAVED FILES:
   • cogaly_xgb_v1.pkl (Main model)
   • cogaly_scaler_v1.pkl (Feature scaler)
   • cogaly_features_v1.pkl (Feature names)
   • cogaly_explainer_v1.pkl (SHAP explainer)
   • cogaly_metrics_v1.pkl (Performance metrics)
""")

print("=" * 80)
print("✅ Ready for production deployment!")
print("=" * 80)

---

## Credits

**Team:** Cogaly  
**Hackathon:** AI 4 Alzheimer's  
**Date:** December 2025

**Disclaimer:** This model is intended for research and educational purposes. It is not approved for clinical diagnosis. Always consult qualified healthcare professionals for medical decisions.